<h1 style="text-align:center">Sangam 2019 - ML Hackathon by IITMAA</h1>
<p>
    <strong>Approach used: </strong>SARIMAX (Seasonal Autoregressive Integrated Moving Average with eXogeneous variables)<br><br>
    <strong>Reason: </strong>The data provided is seasonal, and it is a time series data with multiple exogeneous variables influencing the result. Hence, the optimal statistical model that can be applied to this task is SARIMAX
    <br><br>
    <strong>Main Modules Used: </strong>
    <ul>
        <li><code>statsmodel</code> package in Python</li>
    </ul>
</p>

<h2>Import Required Modules</h2>

In [132]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import statsmodels.api as sm
from tqdm import tqdm

<h2>Read the Train Data</h2>

In [133]:
data = pd.read_csv('Train.csv')
data.index = data.date_time
data = data.drop(['date_time'],axis=1)
data.head()

,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
date_time,,,,,,,,,,,,,,
2012-10-02 09:00:00,None,121,89,2,329,1,1,288.28,0.0,0.0,40,Clouds,scattered clouds,5545
2012-10-02 10:00:00,None,178,67,3,330,1,1,289.36,0.0,0.0,75,Clouds,broken clouds,4516
2012-10-02 11:00:00,None,113,66,3,329,2,2,289.58,0.0,0.0,90,Clouds,overcast clouds,4767
2012-10-02 12:00:00,None,20,66,3,329,5,5,290.13,0.0,0.0,90,Clouds,overcast clouds,5026
2012-10-02 13:00:00,None,281,65,3,329,7,7,291.14,0.0,0.0,75,Clouds,broken clouds,4918


<h2>Data Preprocessing</h2>
<p>For handling categorical variables <code>is_holiday</code>, <code>weather_type</code>, <code>weather_description</code>, we perform <strong>one-hot encoding</strong></p>

In [134]:
def pre_process(data):
    data['holiday'] = 0
    for i in tqdm(range(len(data))):
        if(data.iloc[i]['is_holiday'] != "None"):
            data.iloc[i]['holiday'] = 1
    weather_type = pd.get_dummies(data['weather_type'],prefix="weather_type")
    weather_desc = pd.get_dummies(data['weather_description'],prefix="weather_desc")
    data = data.drop(['weather_type','weather_description','is_holiday'],axis=1)
    data = pd.concat([data,weather_type,weather_desc],axis=1)
    data.head()
    return(data)

In [135]:
data = pre_process(data)

  0%|          | 0/33750 [00:00<?, ?it/s]/Users/hemanthkumar/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
100%|██████████| 33750/33750 [00:16<00:00, 2003.46it/s]


<h2>Train Data Assignment </h2>
<h4>(Here all data is set to train, but for validation the commented out part should be used)</h4>

In [136]:
# train = data.iloc[:int(0.9*len(data))]
# test = data.iloc[int(0.9*len(data)):]'

train = data

<h2>Specify endogenous and exogenous variables in the data</h2>

In [137]:
# Variables
exog_data = train.drop(['traffic_volume'],axis=1)
exog = sm.add_constant(exog_data)
endog = train[[u'traffic_volume']]

print(endog)
print(exog)
# nobs = endog.shape[0]

                     traffic_volume
date_time                          
2012-10-02 09:00:00            5545
2012-10-02 10:00:00            4516
2012-10-02 11:00:00            4767
2012-10-02 12:00:00            5026
2012-10-02 13:00:00            4918
2012-10-02 14:00:00            5181
2012-10-02 15:00:00            5584
2012-10-02 16:00:00            6015
2012-10-02 17:00:00            5791
2012-10-02 18:00:00            4770
2012-10-02 19:00:00            3539
2012-10-02 20:00:00            2784
2012-10-02 21:00:00            2361
2012-10-02 22:00:00            1529
2012-10-02 23:00:00             963
2012-10-03 00:00:00             506
2012-10-03 01:00:00             321
2012-10-03 02:00:00             273
2012-10-03 03:00:00             367
2012-10-03 04:00:00             814
2012-10-03 05:00:00            2718
2012-10-03 06:00:00            5673
2012-10-03 08:00:00            6511
2012-10-03 09:00:00            5471
2012-10-03 12:00:00            5097
2012-10-03 13:00:00         

<h2>Train the Model (Slow Cell)</h2>

In [138]:
# Fit the model
mod = sm.tsa.statespace.SARIMAX(endog, exog=exog, order=(1,0,1))
fit_res = mod.fit(disp=False)
print(fit_res.summary())

/Users/hemanthkumar/anaconda3/lib/python2.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/hemanthkumar/anaconda3/lib/python2.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


                           Statespace Model Results                           
Dep. Variable:         traffic_volume   No. Observations:                33750
Model:               SARIMAX(1, 0, 1)   Log Likelihood             -274916.124
Date:                Sun, 04 Aug 2019   AIC                         549960.248
Time:                        23:55:18   BIC                         550499.560
Sample:                             0   HQIC                        550132.328
                              - 33750                                         
Covariance Type:                  opg                                         
                                                       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
const                                            -2874.4951     99.034    -29.025      0.000   -3068.597   -2680.393
air_pollution_ind

<h2>Read in Test Set</h2>

In [139]:
test_set = pd.read_csv("test.csv")
test_set.index = test_set.date_time
test_set = test_set.drop(['date_time'],axis=1)
test_set.head()

,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description
date_time,,,,,,,,,,,,,
2017-05-18 00:00:00,None,73,63,1,27,4,4,285.15,0.0,0,90,Rain,moderate rain
2017-05-18 00:00:00,None,251,63,1,27,4,4,285.15,0.0,0,90,Mist,mist
2017-05-18 00:00:00,None,75,56,1,0,1,1,285.15,0.0,0,90,Drizzle,light intensity drizzle
2017-05-18 01:00:00,None,98,56,1,351,2,2,284.79,0.0,0,90,Rain,heavy intensity rain
2017-05-18 01:00:00,None,283,56,1,351,1,1,284.79,0.0,0,90,Mist,mist


In [140]:
test_set = pre_process(test_set)

  1%|          | 162/14454 [00:00<00:08, 1616.28it/s]/Users/hemanthkumar/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
100%|██████████| 14454/14454 [00:06<00:00, 2111.87it/s]


In [141]:
test_set.head()

,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,...,weather_desc_shower drizzle,weather_desc_sky is clear,weather_desc_sleet,weather_desc_smoke,weather_desc_snow,weather_desc_thunderstorm,weather_desc_thunderstorm with heavy rain,weather_desc_thunderstorm with light drizzle,weather_desc_thunderstorm with light rain,weather_desc_thunderstorm with rain
date_time,,,,,,,,,,,,,,,,,,,,,
2017-05-18 00:00:00,73,63,1,27,4,4,285.15,0.0,0,90,...,0,0,0,0,0,0,0,0,0,0
2017-05-18 00:00:00,251,63,1,27,4,4,285.15,0.0,0,90,...,0,0,0,0,0,0,0,0,0,0
2017-05-18 00:00:00,75,56,1,0,1,1,285.15,0.0,0,90,...,0,0,0,0,0,0,0,0,0,0
2017-05-18 01:00:00,98,56,1,351,2,2,284.79,0.0,0,90,...,0,0,0,0,0,0,0,0,0,0
2017-05-18 01:00:00,283,56,1,351,1,1,284.79,0.0,0,90,...,0,0,0,0,0,0,0,0,0,0


<h2>Handling columns that aren't present in the test set, but are in the train set</h2>

In [142]:
for i in train.columns:
    if i not in test_set.columns:
        test_set[i] = 0
test_set.tail()

,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,...,weather_desc_thunderstorm with light rain,weather_desc_thunderstorm with rain,traffic_volume,weather_type_Squall,weather_desc_SQUALLS,weather_desc_freezing rain,weather_desc_light rain and snow,weather_desc_shower snow,weather_desc_thunderstorm with drizzle,weather_desc_very heavy rain
date_time,,,,,,,,,,,,,,,,,,,,,
2018-09-30 19:00:00,176,21,4,345,4,4,283.45,0.0,0,75,...,0,0,0,0,0,0,0,0,0,0
2018-09-30 20:00:00,214,95,8,280,6,6,282.76,0.0,0,90,...,0,0,0,0,0,0,0,0,0,0
2018-09-30 21:00:00,173,63,4,238,1,1,282.73,0.0,0,90,...,0,0,0,0,0,0,0,0,0,0
2018-09-30 22:00:00,21,57,8,268,7,7,282.09,0.0,0,90,...,0,0,0,0,0,0,0,0,0,0
2018-09-30 23:00:00,116,70,8,328,6,6,282.12,0.0,0,90,...,0,0,0,0,0,0,0,0,0,0


<h2>Forecasting the <code>traffic_volume</code> for the given test set</h2>

In [143]:
# last_train = train.iloc[len(train)-1].name
first_predict = test_set.iloc[0].name
# print(last_train,first_predict)
# import datetime as dt

# start_dt = dt.datetime.strptime(last_train, '%Y-%m-%d %H:%M:%S')
# predict_dt = dt.datetime.strptime(first_predict, '%Y-%m-%d %H:%M:%S')
# diff = (predict_dt - start_dt) 
# days, seconds = diff.days, diff.seconds
# hours = days * 24 + seconds // 3600
# print(hours)

exog1 = (sm.add_constant(test_set).loc[first_predict:])
exog1 = exog1.drop(['traffic_volume'],axis=1)

# print(pd.concat([exog,exog1]))
# predict = fit_res.predict(start=hours,end=hours,exog=exog1)
# print(predict)

print(exog1)
print(exog1.shape)
forecast = fit_res.forecast(steps = len(test_set),exog = exog1)
print(forecast,len(forecast),len(test_set))

                     const  air_pollution_index  humidity  wind_speed  \
date_time                                                               
2017-05-18 00:00:00    1.0                   73        63           1   
2017-05-18 00:00:00    1.0                  251        63           1   
2017-05-18 00:00:00    1.0                   75        56           1   
2017-05-18 01:00:00    1.0                   98        56           1   
2017-05-18 01:00:00    1.0                  283        56           1   
2017-05-18 02:00:00    1.0                  115        49           1   
2017-05-18 02:00:00    1.0                   48        49           1   
2017-05-18 02:00:00    1.0                  133        49           1   
2017-05-18 03:00:00    1.0                  130        60           2   
2017-05-18 03:00:00    1.0                   93        60           2   
2017-05-18 03:00:00    1.0                  218        60           2   
2017-05-18 04:00:00    1.0                  272    

/Users/hemanthkumar/anaconda3/lib/python2.7/site-packages/statsmodels/tsa/base/tsa_model.py:531: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


(33750    2057.088512
33751    1766.392782
33752    1860.182800
33753    3568.848822
33754    2442.566253
33755    3571.715977
33756    2482.516740
33757    2500.141731
33758    3764.034761
33759    2646.692031
33760    2501.016075
33761    3850.940161
33762    2719.604551
33763    2680.391301
33764    3265.266281
33765    2858.022202
33766    2799.374012
33767    3296.799273
33768    2342.608226
33769    3399.749559
33770    2792.205783
33771    3361.693661
33772    2751.882582
33773    2971.153857
33774    2599.281847
33775    2990.982564
33776    2787.607778
33777    2588.345206
33778    2780.416817
33779    2701.955818
            ...     
48174    4534.238095
48175    4386.120810
48176    4420.523700
48177    4379.193040
48178    4326.763070
48179    4431.559711
48180    4350.956930
48181    4479.656248
48182    4425.213185
48183    4401.658004
48184    4384.655348
48185    4518.834064
48186    4442.959029
48187    4469.569271
48188    2627.904270
48189    4570.759176
48190    266

In [144]:
result_data = pd.DataFrame(index=test_set.index, columns=['date_time','traffic_volume'])
result_data.head()

,date_time,traffic_volume
date_time,,
2017-05-18 00:00:00,NaN,NaN
2017-05-18 00:00:00,NaN,NaN
2017-05-18 00:00:00,NaN,NaN
2017-05-18 01:00:00,NaN,NaN
2017-05-18 01:00:00,NaN,NaN


In [145]:
chk = 0
for i in tqdm(forecast):
    result_data.iloc[chk]["date_time"] = test_set.iloc[chk].name
    result_data.iloc[chk]["traffic_volume"] = i
    chk+=1
result_data.head()

100%|██████████| 14454/14454 [00:12<00:00, 1137.70it/s]


,date_time,traffic_volume
date_time,,
2017-05-18 00:00:00,2017-05-18 00:00:00,2057.09
2017-05-18 00:00:00,2017-05-18 00:00:00,1766.39
2017-05-18 00:00:00,2017-05-18 00:00:00,1860.18
2017-05-18 01:00:00,2017-05-18 01:00:00,3568.85
2017-05-18 01:00:00,2017-05-18 01:00:00,2442.57


In [146]:
result_data.to_csv('results.csv', header=['date_time','traffic_volume'], index=False) 